##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Kisisellestirilmis egitim: Temelleri

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/tr/r1/tutorials/eager/custom_training.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/tr/r1/tutorials/eager/custom_training.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

Bir onceki egitim kitapciginda TensorFLow API'sinin makine ogrenmesinin temel yapi taslarindan biri olan otomatik degisimlerde kullanimini gorduk.
Bu kitapcikta ise daha onceki kitapciklarda ogrendigimiz TensorFlow ilkellerini basit makine ogrenmesinde kullanacagiz.

Bunun disinda TensorFlow icinde soyutlamayi kolaylastiran yuksek seviye sinir agi API'si 'tf.keras'i da bulundurmaktadir. Sinir aglari ile calisanlarin bu yuksek seviye API'larini kullanmalarini siddetle tavsiye ederiz. Fakat, bu kisa kitapcikta saglam temeller olusturmak icin sinir aglari egitiminin ilk ilkelerinden bahsedecegiz.

## Kurulum

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

tf.enable_eager_execution()

## Degiskenler

TensorFlow'daki Tensorlar sabit(immutable), durumsuz(stateless) nesnelerdir. Fakat makine ogrenmesi modellerinin durumu degiskendir: modeliniz egitiminde ilerledikce ayni kodun yaptigi tahminler farkli davranislar sergilemelidir (bunu dusuk kayipla yapmasini umut ediyoruz!). Yaptiginiz islemler suresunce degisen bu durumu Python'un durumsal bir yazilim dili olmasi ile gosterebiliriz:


In [0]:
# Python durumunu kullanmak
x = tf.zeros([10, 10])
x += 2  # This is equivalent to x = x + 2, which does not mutate the original
        # value of x
print(x)

TensorFlow icerisinde durumsal operasyonlar bulundurmaktadir ve cogu zaman durumun gosterimi icin bunlari kullanmak dusuk-seviye Python gosterimlerinden daha iyidir. Ornegin, modeldeki agirliklari gostermek icin TensorFlow degiskenlerini kullanmak hem kolay hem de verimlidir.

TensorFlow hesaplamalarinda kullanildiginda kaydedilen degeri okuyacak olan, Degisken icinde bir deger bulunduran bir nesnedir. TensorFlow degiskenininde kaydedilen degeri degistirmek icin `tf.assign_sub` ve `tf.scatter_update` gibi cesitli operasyonlar bulunmaktadir.

In [0]:
v = tf.Variable(1.0)
assert v.numpy() == 1.0

# Degeri yeniden esleyelim
v.assign(3.0)
assert v.numpy() == 3.0

# tf.square() gibi bir TensorFlow operasyonunda `v`yi kullanmalim ve yeniden esleyelim
v.assign(tf.square(v))
assert v.numpy() == 9.0

Degiskenler kullanilarak yapilan hesaplamalarda egimler otomatik olarak takip edelir. Katistirmalari gosteren Degiskenlerde ise TensorFlow donem donem guncellestirme yapar, ki bunlar hesaplama gucunde ve hafizada verimlidir.

Degiskenlerin kullanilmasi ayni zamanda kodunuzu okuyan kisiye bu durumun sabit olmadigini(mutable) belirtir.

## Ornek: Dogrusal modeli uydurmak

Simdi 'Tensor', 'GradientTape', ve 'Variable' gibi ogrendigimiz kavramlari bir araya getirip basit bir modeli olusturalim ve egitelim. Bu genel olarak birkac asamadan olusur:

1. Modeli tanimla.
2. Kayip fonksiyonunu tanimla.
3. Egitim verilerini topla.
4. Egitim verilerini incele ve "optimizer" kullanarak degiskenleri veriye uydur.

Bu egitimde, onemsiz bir ornek olan `f(x) = x * W + b` dogrusal modelinin ustunden gececegiz. Bu modelde iki degisken bulunmaktadir - `W` ve `b`. Dahasi, veriyi oyle olusturacagiz ki iyi egitilmis bir model `W = 3.0` ve `b = 2.0` degerlerine sahip olacak.

### Modeli tanimlayalim

Degiskenleri ve hesaplamalari kapsayacak basit bir sinif tanimlayalim.

In [0]:
class Model(object):
  def __init__(self):
    # Degiskeni (5.0, 0.0) degerlerine ilkle
    # Normalde bu degerler rasgele verilir.
    self.W = tf.Variable(5.0)
    self.b = tf.Variable(0.0)

  def __call__(self, x):
    return self.W * x + self.b

model = Model()

assert model(3.0).numpy() == 15.0

### Kayip fonksiyonunu tanimlayalim

Kayip fonksiyonu belirli bir girdiye gore beklenen cikti degeri ile modelin sonuclarinin ne kadar benzedigini hesaplar. Burada standart L2 kaybini kullanalim.

In [0]:
def loss(predicted_y, desired_y):
  return tf.reduce_mean(tf.square(predicted_y - desired_y))

### Egitim verisini toplayalim

Biraz gurultulu bir egitim verisi sentezleyelim.

In [0]:
TRUE_W = 3.0
TRUE_b = 2.0
NUM_EXAMPLES = 1000

inputs  = tf.random_normal(shape=[NUM_EXAMPLES])
noise   = tf.random_normal(shape=[NUM_EXAMPLES])
outputs = inputs * TRUE_W + TRUE_b + noise

Modelimizi egitmeye baslamadan once modelimizin durumuna bir bakalim. Modelin tahminlarini kirmizi ile egittim verisini mavi ile cizecegiz.

In [0]:
import matplotlib.pyplot as plt

plt.scatter(inputs, outputs, c='b')
plt.scatter(inputs, model(inputs), c='r')
plt.show()

print('Current loss: '),
print(loss(model(inputs), outputs).numpy())

### Egitim dongusunu tanimlayalim

Simdi agimiz ve egitim verimiz hazir. Haydi modelimizi egitelim. Egitim verilerini kullanarak modelin degiskenlerini (`W` ve `b`) guncelleyelim ki kayip fonksiyonumuzun degeri [alcalan egim](https://en.wikipedia.org/wiki/Gradient_descent) ile azalsin. `tf.train.Optimizer` gerceklemesi alcalan egimin bircok degisimin yakalar. Bu nedenle bu gerceklemeleri kullanmanizi siddetle tavsiye ederiz. Fakat burada ilk ilkeleri kullanarak olusturdugumuz model icin bu basit matematik islemlerini kendimiz gercekleyecegiz.

In [0]:
def train(model, inputs, outputs, learning_rate):
  with tf.GradientTape() as t:
    current_loss = loss(model(inputs), outputs)
  dW, db = t.gradient(current_loss, [model.W, model.b])
  model.W.assign_sub(learning_rate * dW)
  model.b.assign_sub(learning_rate * db)

Son olarak, egitim verisinden tekrar tekrar gecerek 'W' ve 'b' degerlerinin nasil evrimlestigini gorelim.

In [0]:
model = Model()

# W ve b degerlerini daha sonra bir grafikte gostermek icin toplayalim
Ws, bs = [], []
epochs = range(10)
for epoch in epochs:
  Ws.append(model.W.numpy())
  bs.append(model.b.numpy())
  current_loss = loss(model(inputs), outputs)

  train(model, inputs, outputs, learning_rate=0.1)
  print('Epoch %2d: W=%1.2f b=%1.2f, loss=%2.5f' %
        (epoch, Ws[-1], bs[-1], current_loss))

# Simdi bunlari cizelim
plt.plot(epochs, Ws, 'r',
         epochs, bs, 'b')
plt.plot([TRUE_W] * len(epochs), 'r--',
         [TRUE_b] * len(epochs), 'b--')
plt.legend(['W', 'b', 'true W', 'true_b'])
plt.show()


## Siradaki adimlar

Bu egitim kitapciginda `Variable(Degisken)`leri gozden gecirdik ve TensorFlow ilkellerini kullanarak basit dogrusal bir model egittik.

Teoride bunlar TensorFlow'u makine ogrenmesi arastirmalarinda kullanmanizda yeterlidir.
Pratikte, ozellikle sinir aglari icin, yuksek seviye yapi taslari ("layers(katmanlar)"), durumu(state) kaydetme ve geri yukleme imkanlari, cok sayida kayip fonksiyonu ve eniyileme taktikleri sundugu icin `tf.keras` gibi yuksek seviye API kullanimi cok daha kolay olacaktir.
